In [1]:
import pandas as pd
from datasets import load_dataset

# Load the Bengali subset of the dataset
dataset = load_dataset('mteb/IndicSentiment', 'bn')

# Convert to pandas dataframe
df = dataset['train'].to_pandas()  # Assuming you want the training split

# Display the dataframe
df.head()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang
0,Home,Appliances,Home theater,Soundbars,Boat,"Bluetooth/wireless, HDMI, audio output mode, i...",HDMI,This boat's soundbar is still wire-connectivit...,Negative,এই বোটের সাউন্ডবারটি এখনও সব স্পিকারের জন্য তা...,bn
1,Hobbies,Music,Audio Output,headphones,Zeb Paradise,"on-ear, in-ear, wired, bluetooth, earbuds, noi...",Over-ear with mic,Foldable type of microphone with mic and micro...,Positive,মাইক ও মাইক্রো এসডি কার্ড স্লটসহ ভাঁজযোগ্য ধরন...,bn
2,Entertainment,Apps,Social Media,Social networking,Instagram,"find friends, share photos and moments, free m...",daily status,The recently included feature of stories by de...,Positive,24 ঘণ্টার জন্য গল্পের যে বৈশিষ্ট্যটি ডিফল্টভাব...,bn
3,Transportation,Air,Flights,International,Emirates,"luggage allowance, affordable rates, luxury, f...",Rates Luggage allowance,"Rates are competitive, almost always the best ...",Positive,"""দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে ...",bn
4,Home,Appliances,Fan,Exhaust fan,Bajaj Maxima,"remove moisture/unpleasant odour, air delivery...",Front Shutter,Looks very big and efficient. But since there ...,Negative,দেখতে অনেক বড় এবং দক্ষ। কিন্তু যেহেতু সামনে কো...,bn


In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

In [13]:
LOCATION = "us-central1"

In [14]:
vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-1.5-flash")

def get_gemini_output(prompt = "", max_out_tokens = 128, temp = 0.0, top_p = 0.9, top_k = 10):
    responses = model.generate_content(
        contents=prompt,
        generation_config={
            "max_output_tokens": max_out_tokens,
            "temperature": temp,
            "top_p": top_p,
            "top_k": top_k
        },
    stream=True,
    )

    response_list = []
    for response in responses:
        try:
            response_list.append(response.text)
        except IndexError:
            response_list.append("")
            continue
    prompt_response = " ".join(response_list)
    return prompt_response

In [15]:
get_gemini_output("বাংলাদেশের ৫ জন কবির নাম লিস্ট আকারে লিখ।")

'বা ংলাদেশের ৫ জন কবির নাম লিস্ট  আকারে:\n\n1. কাজী নজরুল ইস লাম\n2. রবীন্দ্রনাথ ঠাকুর\n3. মাইকেল মধুসূদন দত্ত\n 4. সুকান্ত ভট্টাচার্য\n5. জীবনানন্দ দাশ \n '

In [16]:
zero_shot = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে।

"""

### Three-shot examples

We took the examples from IndicSentiment dataset (not in the eval. / test split).

In [17]:
three_shot = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে।

### উদাহরণ ১
### রিভিউ:
দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে সেরা।

### অনুভূতি:
পজিটিভ

### উদাহরণ ২
### রিভিউ:
অন কল কানেক্টিভিটি অনেক সময় খুব কম হয়।

### অনুভূতি:
নেগেটিভ

### উদাহরণ ৩
### রিভিউ:
হুক এবং লুপ ডিজাইন সহজেই সেট আপ এবং বহন করা যায়।

### অনুভূতি:
পজিটিভ

### আপনার কাজ

"""

In [18]:
chain_of_thought = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে। রিভিউটির অনুভূতি নির্ধারণ করতে গিয়ে নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: রিভিউ পড়া
রিভিউটি পড়ুন এবং এতে কী বলা হয়েছে তা বোঝার চেষ্টা করুন।

### ধাপ ২: ইতিবাচক বা নেতিবাচক শব্দ চিহ্নিত করা
রিভিউতে ব্যবহৃত ইতিবাচক (ভালো, সন্তুষ্ট, দুর্দান্ত) বা নেতিবাচক (খারাপ, বাজে, নিম্নমানের) শব্দগুলি চিহ্নিত করুন।

### ধাপ ৩: অনুভূতি নির্ধারণ
রিভিউতে ইতিবাচক শব্দ বেশি হলে পজিটিভ এবং নেতিবাচক শব্দ বেশি হলে নেগেটিভ অনুভূতি নির্ধারণ করুন।

### উদাহরণ ১
### রিভিউ:
দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে সেরা।

#### চিন্তাধারা:
- ইতিবাচক শব্দ: প্রতিযোগিতামূলক, সেরা
- অনুভূতি: পজিটিভ

### অনুভূতি:
পজিটিভ

### উদাহরণ ২
### রিভিউ:
অন কল কানেক্টিভিটি অনেক সময় খুব কম হয়।

#### চিন্তাধারা:
- নেতিবাচক শব্দ: অনেক সময় খুব কম
- অনুভূতি: নেগেটিভ

### অনুভূতি:
নেগেটিভ

### উদাহরণ ৩
### রিভিউ:
হুক এবং লুপ ডিজাইন সহজেই সেট আপ এবং বহন করা যায়।

#### চিন্তাধারা:
- ইতিবাচক শব্দ: সহজেই সেট আপ এবং বহন করা যায়
- অনুভূতি: পজিটিভ

### অনুভূতি:
পজিটিভ

### আপনার কাজ


"""

In [73]:
category_translation = {
    'Positive': 'পজিটিভ',
    'Negative': 'নেগেটিভ',
}

### Zero-shot

In [20]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gemini_zs_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = zero_shot + " ### রিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_gemini_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gemini_zs_sentiment_results.parquet')

# Save final progress
df.to_parquet('gemini_zs_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

Error at index 70: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "OTHER"
}
Response:
{
  "candidates": [
    {
      "finish_reason": "OTHER"
    }
  ],
  "usage_metadata": {
    "prompt_token_count": 140,
    "candidates_token_count": 1,
    "total_token_count": 141
  }
}
Error at index 78: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "OTHER"
}
Response:
{
  "candidates": [
    {
      "finish_reason": "OTHER"
    }
  ],
  "usage_metadata": {
    "prompt_token_count": 118,
    "candidates_token_count": 1,
    "total_token_count": 119
  }
}
Error at index 115: Cannot get the response text.
Cannot get the Candidate text.
Response candidate cont

In [21]:
zs_results = pd.read_parquet("gemini_zs_sentiment_results.parquet")
zs_results.head()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
0,Home,Appliances,Home theater,Soundbars,Boat,"Bluetooth/wireless, HDMI, audio output mode, i...",HDMI,This boat's soundbar is still wire-connectivit...,Negative,এই বোটের সাউন্ডবারটি এখনও সব স্পিকারের জন্য তা...,bn,### অনুভূতি: **নেগেটিভ** \n\nর িভিউটিতে বোটের...
1,Hobbies,Music,Audio Output,headphones,Zeb Paradise,"on-ear, in-ear, wired, bluetooth, earbuds, noi...",Over-ear with mic,Foldable type of microphone with mic and micro...,Positive,মাইক ও মাইক্রো এসডি কার্ড স্লটসহ ভাঁজযোগ্য ধরন...,bn,এ ই রিভিউটি **উদাসীন**। \n\nরিভিউটিতে পণ্যের ...
2,Entertainment,Apps,Social Media,Social networking,Instagram,"find friends, share photos and moments, free m...",daily status,The recently included feature of stories by de...,Positive,24 ঘণ্টার জন্য গল্পের যে বৈশিষ্ট্যটি ডিফল্টভাব...,bn,### অনুভূতি: **পজিটিভ** \n
3,Transportation,Air,Flights,International,Emirates,"luggage allowance, affordable rates, luxury, f...",Rates Luggage allowance,"Rates are competitive, almost always the best ...",Positive,"""দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে ...",bn,"অ নুভূতি: **পজিটিভ** \n\n""দ রগুলি প্রতিযোগিতাম..."
4,Home,Appliances,Fan,Exhaust fan,Bajaj Maxima,"remove moisture/unpleasant odour, air delivery...",Front Shutter,Looks very big and efficient. But since there ...,Negative,দেখতে অনেক বড় এবং দক্ষ। কিন্তু যেহেতু সামনে কো...,bn,### অনুভূতি: **নেগেটিভ** \n\nর িভিউটিতে উল্লে...


In [22]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gemini_ts_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = three_shot + " ### রিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_gemini_output(prompt_input, max_out_tokens = 256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gemini_ts_sentiment_results.parquet')

# Save final progress
df.to_parquet('gemini_ts_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

Error at index 1: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "OTHER"
}
Response:
{
  "candidates": [
    {
      "finish_reason": "OTHER"
    }
  ],
  "usage_metadata": {
    "prompt_token_count": 292,
    "candidates_token_count": 1,
    "total_token_count": 293
  }
}
Error at index 15: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "OTHER"
}
Response:
{
  "candidates": [
    {
      "finish_reason": "OTHER"
    }
  ],
  "usage_metadata": {
    "prompt_token_count": 394,
    "candidates_token_count": 1,
    "total_token_count": 395
  }
}
Error at index 22: Cannot get the response text.
Cannot get the Candidate text.
Response candidate conten

In [23]:
ts_results = pd.read_parquet("gemini_ts_sentiment_results.parquet")
ts_results.head()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
0,Home,Appliances,Home theater,Soundbars,Boat,"Bluetooth/wireless, HDMI, audio output mode, i...",HDMI,This boat's soundbar is still wire-connectivit...,Negative,এই বোটের সাউন্ডবারটি এখনও সব স্পিকারের জন্য তা...,bn,অ নুভূতি: **নেগেটিভ** \n\nর িভিউতে বলা হয়েছে ...
1,Hobbies,Music,Audio Output,headphones,Zeb Paradise,"on-ear, in-ear, wired, bluetooth, earbuds, noi...",Over-ear with mic,Foldable type of microphone with mic and micro...,Positive,মাইক ও মাইক্রো এসডি কার্ড স্লটসহ ভাঁজযোগ্য ধরন...,bn,None
2,Entertainment,Apps,Social Media,Social networking,Instagram,"find friends, share photos and moments, free m...",daily status,The recently included feature of stories by de...,Positive,24 ঘণ্টার জন্য গল্পের যে বৈশিষ্ট্যটি ডিফল্টভাব...,bn,অ নুভূতি: **পজিটিভ** \n
3,Transportation,Air,Flights,International,Emirates,"luggage allowance, affordable rates, luxury, f...",Rates Luggage allowance,"Rates are competitive, almost always the best ...",Positive,"""দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে ...",bn,অ নুভূতি: **পজিটিভ** \n
4,Home,Appliances,Fan,Exhaust fan,Bajaj Maxima,"remove moisture/unpleasant odour, air delivery...",Front Shutter,Looks very big and efficient. But since there ...,Negative,দেখতে অনেক বড় এবং দক্ষ। কিন্তু যেহেতু সামনে কো...,bn,অ নুভূতি: **নেগেটিভ** \n\nর িভিউটিতে বলা হয়েছ...


In [84]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet('gemini_cot_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = chain_of_thought + " ### রিভিউ: " + review

    try:
        df.at[i, 'predicted_category'] = get_gemini_output(prompt_input, max_out_tokens = 1024)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet('gemini_cot_sentiment_results.parquet')

# Save final progress
df.to_parquet('gemini_cot_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

Error at index 32: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "OTHER"
}
Response:
{
  "candidates": [
    {
      "finish_reason": "OTHER"
    }
  ],
  "usage_metadata": {
    "prompt_token_count": 755,
    "candidates_token_count": 1,
    "total_token_count": 756
  }
}
Error at index 92: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "SAFETY",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.41270763,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.30197924
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probabi

In [85]:
# Display all columns
pd.set_option('display.max_columns', None)

# Display all rows
pd.set_option('display.max_rows', None)

# Display full text in each cell
pd.set_option('display.max_colwidth', None)

# If you have a large DataFrame and want to see it in a scrollable window
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [86]:
cot_results = pd.read_parquet("gemini_cot_sentiment_results.parquet")
cot_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category,category,category_bn,cleaned_predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, speaker connectivity, power output",Dolby output,IKall has now launched a new home theater system with Dolby output. It is a pleasure indeed to ott on Dolby at home.,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার সিস্টেম চালু করেছে। বাড়িতে ডলবিতে ওটিটি উপভোগ করাটা সত্যিই আনন্দের।,bn,"## রিভিউ বিশ্লেষণ:\n\n**ধাপ ১: রিভিউ পড়া**\n\nরিভি উটি আইকলের নতুন হোম থিয়েটার সিস্টেমের ডলবি আউটপুট সম্পর্কে। এটি বলে যে বাড়িতে ডলবিতে ওটিটি উপভোগ করাটা ""সত্যিই আ নন্দের""।\n\n**ধাপ ২: ইতিবাচক বা নেতিবাচক শব্দ চিহ্নিত করা**\n\n* **ইতিবাচক শব্দ:** আনন্দের \n\n**ধাপ ৩: অনুভূতি নির্ধারণ**\n\nরিভিউতে শুধুমাত্র ইতিবাচক শব্দ ব্যবহার করা হয়েছে। \n\n**অনুভূতি:** পজিটিভ \n",Positive,পজিটিভ,পজিটিভ
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage, audio output mode, Woofer",Wattage,"iKall is giving two 500 Watts speakers in its tower speaker set. Though it sounds like a big number, it's just extra wastage because after 200 W the output doesn't vary a lot.",Negative,"আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি স্পিকার দিচ্ছে। যদিও এটি একটি বড় সংখ্যার মত মনে হয়, এটি কেবল অতিরিক্ত অপচয় কারণ 200 ওয়াট এর পরে আউটপুট খুব বেশি পরিবর্তিত হয় না।",bn,"## রিভিউ বিশ্লেষণ:\n\n**ধাপ ১: রিভিউ পড়া**\n\nরিভি উটি আইকলের টাওয়ার স্পিকার সেটের 500 ওয়াটের দুটি স ্পিকার সম্পর্কে। রিভিউয়ার মনে করেন যে 200 ওয়াটের পরে আ উটপুট খুব বেশি পরিবর্তিত হয় না, তাই 500 ওয়াট অতিরিক্ত অপচয়।\n\n**ধাপ ২: ইতি বাচক বা নেতিবাচক শব্দ চিহ্নিত করা**\n\n* **নেতিবাচক শব্দ:** অতিরিক্ত অপচয়, খুব বেশি পরিবর্তিত হয় না\n\n**ধাপ ৩: অনুভূতি নির্ধারণ**\n\nরিভিউতে নেতিবাচক শব্দ বেশি ব্যবহার করা হয়েছে। \n\n**অনুভূতি:** ন েগেটিভ \n",Negative,নেগেটিভ,নেগেটিভ
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, bar, laundry service, games, air conditioning, banquet/dining hall, various cuisines, family friendly","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to be happy with your phone internet; the air conditioners also do not function properly with the remotes to control temperature often missing. The crooked pathway from the gate to the bungalows and rooms makes it difficult for a wheelchair to travel that path, forcing almost every time to draw the car out of the parking to travel to and from even the restaurant.",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন্টারনেট নিয়ে খুশি থাকতে হবে; এয়ার কন্ডিশনারগুলির তাপমাত্রা নিয়ন্ত্রণ করার জন্য রিমোটগুলির সাথে সঠিকভাবে কাজ করে না এবং যা প্রায়শই অনুপস্থিত থাকে। গেট থেকে বাংলো এবং ঘর পর্যন্ত আঁকাবাঁকা পথের কারণে হুইলচেয়ারে ওই পথ দিয়ে যাতায়াত করা কঠিন হয়ে পড়ে, প্রায় প্রতিবারই গাড়ি পার্কিং থেকে বের করে রেস্তোরাঁতেও যাতায়াত করতে হয়।",bn,"## রিভিউ বিশ্লেষণ:\n\n**ধাপ ১: রিভিউ পড়া**\n\nরিভি উটি রিসোর্টের বিভিন্ন নেতিবাচক দিক উল্লেখ করে। ওয়াইফাই , এয়ার কন্ডিশনার, এবং অ্যাক্সেসিবিলিটি সম্পর্কে সমস্যা উল্ লেখ করা হয়েছে।\n\n**ধাপ ২: ইতিবাচক বা নেতিবাচক শব্দ চিহ্নিত করা**\n\n* **নেতিবাচক শব্দ:** ন েই, খুশি থাকতে হবে, সঠিকভাবে কাজ করে না, অনুপস্থিত থাকে, কঠিন হয়ে পড়ে, প্রায় প্রতিবারই, ব ের করে রেস্তোরাঁতেও যাতায়াত করতে হয়।\n\n**ধাপ ৩: অনুভূতি নির্ধারণ**\n\nরিভিউতে শুধুমাত্র নেতিবাচক শব্দ ব্যবহার করা হয়েছে। \n\n**অনুভূতি:** নেগেটিভ \n",Negative,নেগেটিভ,নেগেটিভ
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, moving, heart breaking",Peformances and Moving,"Well directed, & acted, & excellent cinematography. Several moving twists in the plot.",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটোগ্রাফির উদাহরণ। প্লটে বেশ কিছু টুইস্ট রয়েছে।",bn,"## রিভিউ বিশ্লেষণ:\n\n**ধাপ ১: রিভিউ পড়া**\n\nরিভি উটি একটি সিনেমার সম্পর্কে। এতে ভালো পরিচালনা, অভিনয় এ

In [45]:
def clean_predicted_category(text, category_translation):
    # Check if the text is None
    if text is None:
        return ""
    
    # Check if any category is directly present in the prediction string
    for category in category_translation.values():
        if category in text:
            return category
    
    # If no direct match, extract the text within ** ** and remove spaces
    import re
    match = re.search(r'\*\*([^*]*)\*\*', text)
    if match:
        cleaned_text = match.group(1).replace(" ", "")
        # Check if cleaned text matches any category in Bangla
        for category in category_translation.values():
            if cleaned_text == category:
                return category
    
    # If no match found, return None
    return ""

In [46]:
clean_predicted_category("অ নুভূতি: **নেগেটিভ** \n\nর িভিউতে বলা হয়েছে ...", category_translation)

'নেগেটিভ'

In [47]:
clean_predicted_category("অ নুভূতি: **পজিটিভ** \n\nদ রগুলি প্রতিযোগিতাম...", category_translation)

'পজিটিভ'

In [48]:
clean_predicted_category(None, category_translation)

''

In [78]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer

import numpy as np

def update_cleaned_predicted_category(row):
    if pd.isna(row['cleaned_predicted_category']) or row['cleaned_predicted_category'] == '':
        if row['category_bn'] == 'পজিটিভ':
            return 'নেগেটিভ'
        elif row['category_bn'] == 'নেগেটিভ':
            return 'পজিটিভ'
    return row['cleaned_predicted_category']

def evaluate_classification(df):
    df['category'] = df['LABEL']
    # Map English category to Bangla
    df['category_bn'] = df['category'].map(category_translation)

    # Clean the predicted categories
    df['cleaned_predicted_category'] = df['predicted_category'].apply(clean_predicted_category, args=(category_translation,))

    # Update the cleaned predicted categories
    for index, row in df.iterrows():
        old_value = row['cleaned_predicted_category']
        if pd.isna(old_value) or old_value == '':
            if row['category_bn'] == 'পজিটিভ':
                new_value = 'নেগেটিভ'
            elif row['category_bn'] == 'নেগেটিভ':
                new_value = 'পজিটিভ'
            df.at[index, 'cleaned_predicted_category'] = new_value

    unique_labels = set(df['category_bn'].unique()).union(set(df['cleaned_predicted_category'].unique()))
    if len(unique_labels) > 2:
        raise ValueError(f"Expected binary classification but found multiple labels: {unique_labels}")

    # Calculate metrics
    y_true = df['category_bn']
    y_pred = df['cleaned_predicted_category']
    

    # Ensure the labels are binary
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    if len(lb.classes_) != 2:
        raise ValueError(f"Expected binary classification but found these classes: {lb.classes_}")

    pos_label = lb.classes_[1]  # The positive label

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized)
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics

In [79]:
zs_df = pd.read_parquet("gemini_zs_sentiment_results.parquet")
metrics = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.9103
precision: 0.9692
recall: 0.8400
f1_score: 0.9000
auc_roc: 0.9077
log_loss: 3.2347


In [80]:
ts_df = pd.read_parquet("gemini_ts_sentiment_results.parquet")
metrics = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.8718
precision: 0.9825
recall: 0.7467
f1_score: 0.8485
auc_roc: 0.8672
log_loss: 4.6210


In [87]:
cot_df = pd.read_parquet("gemini_cot_sentiment_results.parquet")
metrics = evaluate_classification(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.8782
precision: 0.8684
recall: 0.8800
f1_score: 0.8742
auc_roc: 0.8783
log_loss: 4.3899
